In [1]:
import gym
import random
import numpy as np
from gym.envs.registration import registry, register
import time
from IPython.display import clear_output

In [2]:
try:
    register(
        id='FrozenLakeNoSlip-v0',
        entry_point='gym.envs.toy_text:FrozenLakeEnv',
        kwargs={'map_name' : '4x4', 'is_slippery' : False},
        max_episode_steps=100,
        reward_threshold=0.70, # optimum = 0.74
    )
except:
    pass

env_name = 'FrozenLakeNoSlip-v0'
env = gym.make(env_name)
print(env.observation_space)
print(env.action_space)

Discrete(16)
Discrete(4)


In [3]:
class Agent():
    def __init__(self, env):
        self.is_discrete = type(env.action_space == gym.spaces.discrete.Discrete)
        
        if self.is_discrete:
            self.action_size = env.action_space.n
            print("action size", self.action_size)
        else:
            self.action_low = env.action_space.low
            self.action_high = env.action_space.high
            self.action_shape = env.action_space.shape
            print("action range", self.action_low, self.action_high)
    
    def get_action(self, state):
        if self.is_discrete:
            action = random.choice(range(self.action_size))
        else:
            action = np.random.uniform(self.action_low, 
                                       self.action_high, 
                                       self.action_shape)
#         pole_angle = state[2]
#         action = 0 if pole_angle<0 else 1
        return action

In [4]:
class QAgent(Agent):
    def __init__(self, env, discount_rate=0.97, learning_rate=0.01):
        super().__init__(env)
        self.state_size = env.observation_space.n
        print("state size", self.state_size)
        
        self.eps = 1.0
        self.discount_rate = discount_rate
        self.learning_rate = learning_rate
        self.build_model()
        
    def build_model(self):
        self.q_table = 1e-3*np.random.random([self.state_size, self.action_size])
        
    def get_action(self, state):
        q_state = self.q_table[state]
        action_greedy = np.argmax(q_state)
        action_random = super().get_action(state)        
        return action_random if random.random() < self.eps else action_greedy
    
    def train(self, experience):
        state, action, next_state, reward, done = experience
        
        q_next = self.q_table[next_state]
        q_next = np.zeros([self.action_size]) if done else q_next
        q_target = reward + self.discount_rate * np.max(q_next)
        
        q_update = q_target - self.q_table[state, action]
        self.q_table[state, action] += self.learning_rate * q_update        
        
        if done:
            self.eps = self.eps * 0.99
        
agent = QAgent(env)        

action size 4
state size 16


In [6]:
total_reward = 0
for ep in range(100):
    state = env.reset()
    done = False
    while not done:
        action = agent.get_action(state)
        next_state, reward, done, info = env.step(action)
        agent.train((state, action, next_state, reward, done))
        state = next_state
        total_reward += reward
        
        print("s", state, "a", action)
        print("Episode: {}, Total Reward: {}, eps: {}".format(ep, total_reward, agent.eps))
        env.render()
        print(agent.q_table)
        time.sleep(0.05)
        clear_output(wait=True)
    
env.close()

s 15 a 2
Episode: 99, Total Reward: 48.0, eps: 0.13397967485796175
  (Right)
SFFF
FHFH
FFFH
HFFG
[[7.06787525e-04 7.52856013e-04 6.39237940e-04 7.06927837e-04]
 [4.03134309e-04 5.58907557e-04 7.12448508e-04 7.31712535e-04]
 [1.98609591e-04 6.37157424e-04 4.07069324e-04 8.26995486e-04]
 [2.18117219e-04 6.57392860e-04 8.38860796e-04 1.53956000e-04]
 [3.15947664e-04 1.05294485e-03 3.75956663e-04 7.07210229e-04]
 [2.20652194e-04 3.21300820e-04 4.97606661e-04 6.86243223e-04]
 [7.00551277e-04 1.78452644e-03 6.83674025e-04 3.90126134e-04]
 [9.44295021e-04 2.94333650e-04 2.49445488e-04 4.38242770e-04]
 [9.14487999e-04 1.59599776e-05 2.56480556e-03 7.71272565e-04]
 [9.19584580e-04 1.02175167e-03 1.55163760e-02 1.64438586e-04]
 [3.40150162e-04 9.77048134e-02 6.18651558e-04 8.58977657e-04]
 [2.32799641e-04 9.56152575e-04 4.37810113e-04 1.86113781e-04]
 [1.05832898e-04 4.03795529e-04 8.80179189e-04 3.97586193e-04]
 [8.64377047e-04 5.04696082e-04 2.04974461e-02 9.24279697e-04]
 [5.16842255e-04 2.16